# TableauSimplex

In [1]:
#using Pkg; Pkg.add("https://github.com/krislock/TableauSimplex.jl.git")

In [2]:
using TableauSimplex

# Nodes/Vertices
V = [1, 2, 3, 4, 5, 6]

# Arcs/Edges
E = [(1,2), (1,3), (1,5), (2,3), (2,4), (3,4), (3,5), (4,6), (5,6)]

# Demands
b = [-1, -3, 0, 0, 0, 4]

# Costs
c = [2, 3, 3, 2, 4, 1, 2, 3, 1] 

m, n = length(V), length(E)

(6, 9)

# Phase I

In [3]:
# Add artificial node m+1 with directed edges 
# from vertices with negative demands and to 
# vertices with positive demands

E0 = copy(E)
for i in V
    if b[i] < 0
        push!(E0, (i, m+1))
    else
        push!(E0, (m+1, i))
    end
end
E0

15-element Vector{Tuple{Int64, Int64}}:
 (1, 2)
 (1, 3)
 (1, 5)
 (2, 3)
 (2, 4)
 (3, 4)
 (3, 5)
 (4, 6)
 (5, 6)
 (1, 7)
 (2, 7)
 (7, 3)
 (7, 4)
 (7, 5)
 (7, 6)

In [4]:
# Create the incidence matrix A0

A0 = zeros(eltype(b), m+1, length(E0))
for (k, e) in enumerate(E0)
    i, j = e
    A0[i,k] = -1
    A0[j,k] = 1
end
b0 = [b; 0]
[A0 b0]

7×16 Matrix{Int64}:
 -1  -1  -1   0   0   0   0   0   0  -1   0   0   0   0   0  -1
  1   0   0  -1  -1   0   0   0   0   0  -1   0   0   0   0  -3
  0   1   0   1   0  -1  -1   0   0   0   0   1   0   0   0   0
  0   0   0   0   1   1   0  -1   0   0   0   0   1   0   0   0
  0   0   1   0   0   0   1   0  -1   0   0   0   0   1   0   0
  0   0   0   0   0   0   0   1   1   0   0   0   0   0   1   4
  0   0   0   0   0   0   0   0   0   1   1  -1  -1  -1  -1   0

In [5]:
table = Rational{Int}[ zeros(1, n) ones(1, size(A0,2)-n) 0; A0[1:end-1,:] b0[1:end-1] ]
basis = collect(n+1:n+m)
varnames = ["x$i$j" for (i,j) in E]
artificials = ["a$i" for i in V]
T1 = Tableau(table, basis, [varnames; artificials])


         x12    x13    x15    x23    x24    x34    x35    x46    x56     a1     a2     a3     a4     a5     a6
-----------------------------------------------------------------------------------------------------------------------
Z   |                                                                     1      1      1      1      1      1 |       
-----------------------------------------------------------------------------------------------------------------------
a1  |     -1     -1     -1                                               -1                                    |     -1
a2  |      1                   -1     -1                                        -1                             |     -3
a3  |             1             1            -1     -1                                  1                      |       
a4  |                                  1      1            -1                                  1               |       
a5  |                    1                      

In [6]:
clean!(T1)


         x12    x13    x15    x23    x24    x34    x35    x46    x56     a1     a2     a3     a4     a5     a6
-----------------------------------------------------------------------------------------------------------------------
Z   |            -2     -2     -2     -2                                                                       |     -8
-----------------------------------------------------------------------------------------------------------------------
a1  |      1      1      1                                                1                                    |      1
a2  |     -1                    1      1                                         1                             |      3
a3  |             1             1            -1     -1                                  1                      |       
a4  |                                  1      1            -1                                  1               |       
a5  |                    1                      

In [7]:
simplex!(T1, verbose=true);


         x12    x13    x15    x23    x24    x34    x35    x46    x56     a1     a2     a3     a4     a5     a6
-----------------------------------------------------------------------------------------------------------------------
Z   |            -2     -2     -2     -2                                                                       |     -8
-----------------------------------------------------------------------------------------------------------------------
a1  |      1      1      1                                                1                                    |      1
a2  |     -1                    1      1                                         1                             |      3
a3  |             1             1            -1     -1                                  1                      |       
a4  |                                  1      1            -1                                  1               |       
a5  |                    1                      

In [8]:
println("Basic feasible solution:")
optsoln = [E0[T1.basis] Int.(rhs(T1))]

Basic feasible solution:


6×2 Matrix{Any}:
 (1, 2)  1
 (4, 6)  4
 (1, 3)  0
 (2, 4)  4
 (1, 5)  0
 (7, 6)  0

# Phase II

In [9]:
# Drop all artificial variables and use original edge costs
inds = [i for i=1:m if T1.basis[i] <= n]
table = [
    c' 0
    T1.table[inds.+1, 1:n] T1.table[inds.+1, end]
]
basis = T1.basis[inds]
T2 = Tableau(table, basis, varnames)


         x12    x13    x15    x23    x24    x34    x35    x46    x56
-----------------------------------------------------------------------------
Z   |      2      3      3      2      4      1      2      3      1 |       
-----------------------------------------------------------------------------
x12 |      1                   -1             1                    1 |      1
x46 |                                                       1      1 |      4
x13 |             1             1            -1     -1               |       
x24 |                                  1      1                    1 |      4
x15 |                    1                           1            -1 |       

In [10]:
clean!(T2)


         x12    x13    x15    x23    x24    x34    x35    x46    x56
-----------------------------------------------------------------------------
Z   |                           1            -2      2            -5 |    -30
-----------------------------------------------------------------------------
x12 |      1                   -1             1                    1 |      1
x46 |                                                       1      1 |      4
x13 |             1             1            -1     -1               |       
x24 |                                  1      1                    1 |      4
x15 |                    1                           1            -1 |       

In [11]:
simplex!(T2, verbose=true);


         x12    x13    x15    x23    x24    x34    x35    x46    x56
-----------------------------------------------------------------------------
Z   |                           1            -2      2            -5 |    -30
-----------------------------------------------------------------------------
x12 |      1                   -1             1                    1 |      1
x46 |                                                       1      1 |      4
x13 |             1             1            -1     -1               |       
x24 |                                  1      1                    1 |      4
x15 |                    1                           1            -1 |       

x56 enters, x12 leaves

         x12    x13    x15    x23    x24    x34    x35    x46    x56
-----------------------------------------------------------------------------
Z   |      5                   -4             3      2               |    -25
--------------------------------------------------------

In [12]:
println("Optimal solution:")
optsoln = [E[T2.basis] Int.(rhs(T2))]

Optimal solution:


5×2 Matrix{Any}:
 (5, 6)  4
 (3, 5)  3
 (2, 3)  3
 (3, 4)  0
 (1, 5)  1